# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel
%nuclio config spec.build.baseImage = "docker-registry.default-tenant.app.groupwaretech.iguazio-c0.com:80/fastai"

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable
%nuclio: setting spec.build.baseImage to 'docker-registry.default-tenant.app.groupwaretech.iguazio-c0.com:80/fastai'


In [3]:
%%nuclio cmd -c
python -m pip install numpy cloudpickle v3io sklearn

In [4]:
import os
from os import environ
from cloudpickle import load
import numpy as np
from typing import List
from datetime import datetime
import mlrun
import fastai
from fastai.text import *
from fastai.callbacks import *

In [5]:
class ClassifierModel(mlrun.runtimes.MLModelServer):
    def __init__(self, name: str, model_dir: str):
        super().__init__(name, model_dir)
        self.DATA_CLAS_PATH = environ['DATA_CLAS_PATH']
        self.MODEL_PATH = environ['MODEL_PATH']
        self.NUM_PREDS = int(environ['NUM_PREDS'])
        self.loaded = False
            
    def load(self):
        """Bypass loading here due to fastai error (prints pretrained model
        to console which mlrun interprets as an error)"""
        pass
    
    def load_model(self):
        """Load model"""
        self.data_clas = load_data("", self.DATA_CLAS_PATH, bs=32)
        self.learn_clas = text_classifier_learner(self.data_clas, AWD_LSTM)
        self.model = self.learn_clas.load(self.MODEL_PATH)

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            # Load model
            if not self.loaded:
                self.loaded = True
                self.load_model()
            
            # Model prediction
            text = body['instances']
            pred = self.model.predict(text)
            
            idxs = []
            pred_labels = []
            
            # Sort predictions by confidence
            preds = list(np.argsort(pred[2]))[::-1]
            
            # Get top predictions
            for p in preds[:self.NUM_PREDS]:
                idxs.append(p.item())
            for idx in idxs:
                pred_labels.append(self.data_clas.classes[idx])
            
            result: np.ndarray = np.asarray(pred_labels)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
        return resp

In [6]:
# nuclio: end-code

In [7]:
from mlrun import new_model_server
fn = new_model_server('model-server', model_class='ClassifierModel', workers=1)
fn.spec.description = "nlp fastai model server"
fn.metadata.categories = ['serving', 'ml']
fn.metadata.labels = {'author': 'nschenone'}
# fn.set_envs({'INFERENCE_STREAM': 'users/nschenone/tststream'})
fn.export("../yaml/model_server.yaml")

> 2020-08-13 19:02:11,830 [info] function spec saved to path: ../yaml/model_server.yaml
